In [ ]:
import ipywidgets as widgets
BIAFLOW_SERVERS=["biaflows.neubias.org"]
connectionDialog_title = widgets.HTML(
    value="<h2>Connect to BIAFLOWS</h2>",
)
connectionDialog_menu = widgets.Dropdown(
       options=BIAFLOW_SERVERS,
       value=BIAFLOW_SERVERS[0],
       description='server:')
connectionDialog_publicKey=widgets.Password(description='public key:')
connectionDialog_privateKey=widgets.Password(description='private key:')
connectionDialog = widgets.VBox([connectionDialog_title, connectionDialog_menu, 
                                 connectionDialog_publicKey, connectionDialog_privateKey])
display(connectionDialog)


In [ ]:
import logging
from cytomine import Cytomine
 
host = connectionDialog_menu.value
publicKey = connectionDialog_publicKey.value
privateKey = connectionDialog_privateKey.value
  
with Cytomine(host=host, public_key=publicKey, private_key=privateKey,
              verbose=logging.INFO) as conn:
    print(conn.current_user)

In [ ]:
from cytomine.models import ProjectCollection
projects = ProjectCollection().fetch()
projectNames = [project.name for project in projects]
projectIDs = [project.id for project in projects]
projectDialog_menu = widgets.Dropdown(
       options=list(zip(projectNames, projectIDs)),
       value=projectIDs[0],
       description='project:')

projectDialog_menu

In [ ]:
%%capture
from neubiaswg5.helpers import NeubiasJob, prepare_data

projectID = projectDialog_menu.value
project = Project().fetch(id=projectID)
argv = ['--cytomine_public_key', publicKey, 
 '--cytomine_host',  host,
 '--cytomine_private_key',  privateKey,
 '--cytomine_id_project',  str(projectID), 
 '--cytomine_id_software', '32392501']
with NeubiasJob.from_cli(argv) as nj:
    jobID = nj.job.id
    in_images, gt_images, in_path, gt_path, out_path, tmp_path = prepare_data(project.disciplineShortName, nj, is_2d=False, **nj.flags)

In [ ]:
print(in_path)
!ls -alh $in_path

In [ ]:
import os
import tifffile
files = [i.path for i in os.scandir(in_path) if i.is_file()]
in_paths_sources = [i for i in files if ('.tif' in i) ]
sources = []
for path in in_paths_sources:
    source = tifffile.imread(path)
    tifffile.imshow(source, cmap='gray')
    sources.append(source)